# IF IT IS YOUR FIRST TIME OPENING THIS FILE, READ CAREFULLY THE <font color='red'>INSTRUCTIONS IN RED</font>

## Requirement: to run the code you must have a <font color='green'>Google Account</font>

## Version 0.07 - 16/01/24

In [ ]:
# @title PIP install
%%capture
!pip install gspread==5.11.2
!pip install gspread_formatting==1.1.2
!pip install geoguessr_async -U
!pip install authlib
!pip install pycountry

In [ ]:
# @title Imports
import gspread
from gspread import Cell
from gspread_formatting import *
from gspread_formatting import batch_updater
from gspread import Client

from IPython.display import Image

from google.colab import auth
from google.auth import default

from authlib.integrations.requests_client import AssertionSession

import os, sys
from datetime import date, datetime

import csv
import json
import requests, io

import asyncio
from geoguessr_async import Geoguessr

In [ ]:
# @title Google authorization for Google Sheets modification (you need to accept it in order to modify the G.Sheets)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# PUT YOUR <font color='red'>NCFA COOKIE</font> IN THE CELL BELOW
*See explanatory GIF below or*

To get your NCFA Cookie by:  


1.   Logging into your Geoguessr account.
2.   Opening your dev tools (press `Ctrl+Shift+I`) in your navigator.
3.   Go to `Application/Storage/Cookies` and copy the value of `_ncfa`.  
Beware you'll have 2 `_ncfa` cookies if you also have a Seterra account.
4.   In the cell below, replace `YOUR_NCFA_COOKIE` (between the quotation marks) by the value you copied.
It should look like this:  
`ncfa_cookie="KAQzsU6Y2JNrie1BcQ5G2tS2Xcmn%2FfWxrm347K0wRvQ%3DyXC4L44k2Hm62b5xTIC%2BLCOP5pdh%2Brl6LXtjOxc0u2zMpqXnfATHU%2BhbLjN7kdyOPRGJma6LvrSG0tHjK9Xvqcax%2FWxOJgva1us6ab5Fq57%8A"`



In [ ]:
ncfa_cookie = "YOUR_NCFA_COOKIE"
client = Geoguessr(ncfa_cookie)

In [ ]:
# @title GIF to get NCFA Cookie in Chrome
Image(url="https://drive.google.com/uc?export=view&id=1JF34gQ_8MDlvMlV5cj1b5rDendqSRnhD")

In [ ]:
# @title GIF to get NCFA Cookie in Firefox
Image(url="https://drive.google.com/uc?export=view&id=1IC4l7MkroKAGTWES-sYUsSySYHFe_d4w")

# PUT YOUR <font color='red'>GOOGLE SHEET URL</font> IN THE CELL BELOW

In the cell below, replace `SHEET_URL` by <font color='red'>**YOUR**</font> Google Sheet URL (between the quotation marks) without the `edit#gid=...` part.  
Your URL should look something like this:  
`"https://docs.google.com/spreadsheets/d/1QbDtbwlBBeaTgqcQJ3EswdFR5IBsNbc5DbQZsKkGOAo"`


In [ ]:
# Google Sheets URL to modify according to yours.
sheet_URL = "SHEET_URL"

# <font color='red'>**FOLLOW THE INSTRUCTIONS IN THE GOOGLE SHEETS CAREFULLY TO KNOW WHEN TO RUN THE CODE. DO NOT RUN IT BEFORE BEING TOLD TO.**</font>

# To run the entire code, click on `Runtime` on the menu at the top and then `Run all` (or press `Ctrl + F9`).
# Then follow instructions.

In [ ]:
# @title Utils functions
from datetime import datetime
import pycountry

def val_to_col(column_int):
  start_index = 1
  letter = ''
  while column_int > 25 + start_index:
    letter += chr(65 + int((column_int-start_index)/26) - 1)
    column_int = column_int - (int((column_int-start_index)/26))*26
  letter += chr(65 - start_index + (int(column_int)))
  return letter

def col_to_val(col):
  return sum([(26**(len(col)-i-1)) * (ord(letter) - ord('A')+1) for i, letter in enumerate(col)])

def alpha2ToCountryname(alpha2: str):
  if alpha2!="" and alpha2!=None:
    tmp = pycountry.countries.get(alpha_2=alpha2)
    if tmp != None:
      return pycountry.countries.get(alpha_2=alpha2).name
  return "Unknown"

def computeGuessTime(start: str, end: str):
  # 2023-11-19T03:32:02.941+00:00
  start = start[:19]
  end = end[:19]
  datetime_start = datetime.strptime(start, '%Y-%m-%dT%H:%M:%S')
  datetime_end = datetime.strptime(end, '%Y-%m-%dT%H:%M:%S')
  return (datetime_end-datetime_start).total_seconds()

def print_players(players: list):
  print("Liste des joueurs, en choisir un :")
  for p in players:
    print("- "+p)

In [ ]:
# @title Function to extract useful infos from duels

async def extractInfosDuels(duel_url, myId):
  datas = []
  duel_infos = await client.get_duel_info(duel_url)
  nb_round = duel_infos.currentRoundNumber

  for i, t in enumerate(duel_infos.teams):
    if t['players'][0]["playerId"] == myId:
      myIdx = i
      if t['players'][0]["progressChange"] != None:
        myRatingBefore = t['players'][0]["progressChange"]["competitiveProgress"]["ratingBefore"]
        myRatingAfter  = t['players'][0]["progressChange"]["competitiveProgress"]["ratingAfter"]
      else:
        myRatingBefore = t['players'][0]['rating']
        myRatingAfter  = myRatingBefore
  opponentIdx = 1-myIdx
  opponentId = duel_infos.teams[opponentIdx]['players'][0]["playerId"]
  opponentInfo = await client.get_user_infos(opponentId)
  opponentUsername = opponentInfo.nick if opponentInfo != None else "Banned"
  opponentCountry = alpha2ToCountryname(duel_infos.teams[opponentIdx]['players'][0]["countryCode"]) if opponentInfo != None else "Unknown"
  opponentRating = duel_infos.teams[opponentIdx]['players'][0]['rating']

  hasWon = True if(myRatingAfter > myRatingBefore) else False
  datas.extend([opponentCountry, opponentId, opponentUsername, opponentRating, myRatingBefore, myRatingAfter, hasWon])

  for i in range(nb_round):
    country = alpha2ToCountryname(duel_infos.rounds[i]["panorama"]["countryCode"])
    startRound = duel_infos.rounds[i]["startTime"]

    guessOpponent = duel_infos.teams[opponentIdx]["roundResults"][i]["bestGuess"]
    distanceOpponent = round(guessOpponent["distance"])/1000 if guessOpponent != None else "No Guess"

    guessMe = duel_infos.teams[myIdx]["roundResults"][i]["bestGuess"]
    if guessMe != None:
      distanceMe = round(guessMe["distance"])/1000
      durationGuessMe = computeGuessTime(startRound, guessMe["created"])
    else:
      distanceMe = "No Guess"
      durationGuessMe = "No Guess"
    datas.extend([country, distanceOpponent, distanceMe, durationGuessMe])

  return datas

def filter_duel_afterdate(lstDuel, date):
  start_date = datetime.strptime(date, "%d-%m-%Y %H:%M:%S")
  lstFiltered = []
  for duel in lstDuel:
    if start_date < datetime.strptime(duel[0], "%d-%m-%Y %H:%M:%S"):
      lstFiltered.append(duel)
  return lstFiltered

In [ ]:
# @title Function to update opponents current ELO

async def update_opponent_ELO(sh):
  batchSize = 50

  now = datetime.now().strftime("%d/%m/%y %H:%M:%S")
  wsOpponent = sh.worksheet("Opponents_ELO")
  ID_rg = "A3:A"
  opponentIDs = [p[0] for p in wsOpponent.get(ID_rg)]

  if len(opponentIDs) == 0:
    print("No opponents to import.")
    sys.exit()

  opponent, cellList = None, []
  for i, opponentID in enumerate(opponentIDs):
    row = i+3
    opponent = await client.get_user_infos(opponentID)
    if opponent:
      opponentName = opponent.nick
      opponentELO = "Banned" if opponent.isBanned else opponent.competitiveRating
    else:
      opponentName, opponentELO = "Banned", "Banned"

    cellList.append(Cell(row, 2, opponentName))
    cellList.append(Cell(row, 3, opponentELO))

    if i%batchSize == batchSize - 1:
      wsOpponent.update_cells(cellList, value_input_option='USER_ENTERED')
      cellList = []

    opponent = None

  if len(cellList) != 0:
    wsOpponent.update_cells(cellList, value_input_option='USER_ENTERED')

  # Update last_update in B1
  wsOpponent.update_cell(1, 2, now)
  return

In [ ]:
# @title Main code
sh = gc.open_by_url(sheet_URL)
wsinfo = sh.worksheet("Player_list")

importBatchSize = 20 # You can modify this value.

player_rg = f"A2:A"
players = [p[0] for p in wsinfo.get(player_rg)]

if len(players) == 0:
  print(f"No player to import. Exiting.")
  sys.exit()
elif len(players) == 1:
  player = players[0]
else:
  print(f"Error, do not use this code if you have more than one player, use the method with Tweaks (see Sheet). Exiting.")
  sys.exit()

wsName = "Data_duel_" + player
try:
  ws_duel = sh.worksheet(wsName)
except:
  print(f"The sheet {wsName} doesn't exist. You need to create it. Exiting.")
  sys.exit()

await client.get_all_my_infos()
Id = client.id

lstDuels = await client.get_ranked_duel_activity()
firstImport = ws_duel.acell("A3").value
if firstImport != "":
  lastDate = ws_duel.acell("F1").value
  lstDuels = filter_duel_afterdate(lstDuels, lastDate)

rowFrom = int(ws_duel.acell("G1").value)

cell_list_duel = []
for i, duel in enumerate(lstDuels):
  row = i + rowFrom
  time, link = duel
  duelDatas = await extractInfosDuels(link, Id)
  cell_list_duel.append(Cell(row, 1, time))
  cell_list_duel.append(Cell(row, 2, link))
  for j, val in enumerate(duelDatas):
    cell_list_duel.append(Cell(row, 4+j, val))
  if i % importBatchSize == importBatchSize - 1:
    ws_duel.update_cells(cell_list_duel, value_input_option='USER_ENTERED')
    cell_list_duel = []
if len(cell_list_duel) != 0:
  ws_duel.update_cells(cell_list_duel, value_input_option='USER_ENTERED')

await update_opponent_ELO(sh)